# BentoML PyTorch MNIST Tutorial

Link to source code: https://github.com/bentoml/BentoML/tree/main/examples/pytorch_yolov5_torchhub/

Install required dependencies:

In [ ]:
%pip install -r requirements.txt

## Load the pre-trained model from Torch Hub

take [`ultralytics/yolov5`](https://github.com/ultralytics/yolov5) as the example

In [ ]:
!git clone 'https://github.com/ultralytics/yolov5.git'

In [ ]:
import torch

# Model
original_model = torch.hub.load("./yolov5", "yolov5s", pretrained=True, source="local")


class WrapperModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, imgs):
        outputs = self.model(imgs)

        # convert outputs to a json serializable list
        results = []
        for det in outputs.pred:
            detections = []
            for i in det:
                d = {}
                d["obj"] = outputs.names[int(i[5])]
                d["position"] = i[:4].tolist()
                d["prob"] = i[4].tolist()
                detections.append(d)
            results.append(detections)

        return results


model = WrapperModel(original_model)

In [ ]:
import PIL.Image
import numpy as np

In [ ]:
# Images
imgs = [
    np.array(PIL.Image.open("yolov5/data/images/bus.jpg")),
    np.array(PIL.Image.open("yolov5/data/images/zidane.jpg")),
]  # batch of images

model(imgs)

## Training and Saving the model

Then we define a simple PyTorch network and some helper functions

### saving the model with some metadata

In [ ]:
import bentoml

bentoml.pytorch.save_model(
    "pytorch_yolov5",
    model,
    signatures={"__call__": {"batchable": True, "batchdim": 0}},
)

## Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

Even though we have only one model, we can create as many api endpoints as we want. Here we create two end points `predict_ndarray` and `predict_image`

In [ ]:
%%writefile service.py

import sys
import os
import typing as t

import numpy as np
import PIL.Image

import bentoml
from bentoml.io import Image
from bentoml.io import JSON


yolo_runner = bentoml.pytorch.get("pytorch_yolov5").to_runner()

svc = bentoml.Service(
    name="pytorch_yolo_demo",
    runners=[yolo_runner],
)


sys.path.append('yolov5')

@svc.api(input=Image(), output=JSON())
async def predict_image(img: PIL.Image.Image) -> list:
    assert isinstance(img, PIL.Image.Image)
    return await yolo_runner.async_run([np.array(img)])


Start a dev model server to test out the service defined above

In [ ]:
!bentoml serve service.py:svc

Now you can use something like:

`curl -H "Content-Type: multipart/form-data" -F'fileobj=@yolov5/data/images/bus.jpg;type=image/png' http://127.0.0.1:3000/predict_image`
    
to send an image to the digit recognition service

## Build a Bento for distribution and deployment

Starting a dev server with the Bento build:

In [ ]:
!bentoml build

In [ ]:
!bentoml serve pytorch_yolo_demo:latest